In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
from PIL import Image
import os

%matplotlib inline

In [2]:
original_images = []
addresses = ['usps_1.jpg', 'usps_2.jpg', 'usps_3.jpg', 'usps_4.jpg', 'usps_5.jpg']

for i in range(len(addresses)):
    original_images.append(plt.imread(addresses[i]))

In [3]:
row = int(original_images[0].shape[0] / 16)
column = int(original_images[0].shape[1] / 16)

print("r: ", row, " column: ", column)

r:  34  column:  33


In [4]:
d = 16
X_list = []
for img in original_images:
    for i in range(row):
        for j in range(column):
            img_digit_one = img[i * d: (i + 1) * d, j * d: (j + 1) * d]
            x = img_digit_one.reshape(1,-1)
            if not ((i > 11) and (j == column - 1)):
                X_list.append(x)
X = np.concatenate(X_list, axis=0)
print(X[0].shape)

(256,)


In [5]:
def L2_norm(A, B):
    norm = 0.0
    n = len(A)

    for i in range(n):
        norm += (A[i] - B[i]) ** 2

    return norm

In [6]:
def closest_centroid(X, centroids):
    """
    Computes the centroid memberships for every example

    Args:
        X (ndarray): (m, n) Input values
        centroids (ndarray): (K, n) centroids

    Returns:
        idx (array_like): (m,) closest centroids

    """
    idx = np.zeros(X.shape[0], dtype = int)

    for i in range(X.shape[0]):
        min_distance = math.inf
        cluster = -1
        for j in range(centroids.shape[0]):
            L2_norm = np.linalg.norm(X[i] - centroids[j])
            if L2_norm < min_distance:
                min_distance = L2_norm
                cluster = j
        idx[i] = cluster

    return idx

In [7]:
def compute_centroids(X, idx, K):
    """
    Returns the new centroids by computing the means of the
    data points assigned to each centroid.

    Args:
        X (ndarray):   (m, n) Data points
        idx (ndarray): (m,) Array containing index of closest centroid for each
                       example in X. Concretely, idx[i] contains the index of
                       the centroid closest to example i
        K (int):       number of centroids

    Returns:
        centroids (ndarray): (K, n) New centroids computed
    """
    centroids = np.zeros((K, X.shape[1]))

    for k in range(K):
        points_cluster_k = X[idx == k]
        centroids[k] = np.mean(points_cluster_k, axis=0)

    return centroids

In [8]:
def k_means(X, initial_centroids, num_iterations):
    """
    This functions clusters inputs

    Args:
        X (ndarray):    (m, n) Data points
        initial_centroids(ndarray):  (k, n)
        num_iterations(int):    max iterations to run k-means

    Returns:
        idx(ndarray):   (m,)  shows id of each x's cluster
        centroids(ndarray): (k, n)  centroids computed
    """
    m, n = X.shape
    K = initial_centroids.shape[0]
    centroids = copy.deepcopy(initial_centroids)
    idx = np.ndarray((m,))

    for i in range(num_iterations):
        idx = closest_centroid(X, centroids)
        new_centroids = compute_centroids(X, idx, K)
        if np.array_equal(new_centroids, centroids):
            break
        centroids = np.copy(new_centroids)

    return idx, centroids

In [9]:
def kMeans_init_centroids(X, K):
    """
    This function initializes K centroids that are to be
    used in K-Means on the dataset X

    Args:
        X (ndarray): Data points
        K (int):     number of centroids/clusters

    Returns:
        centroids (ndarray): Initialized centroids
    """

    randidx = np.random.permutation(X.shape[0])
    centroids = X[randidx[:K]]

    return centroids

In [10]:
def show(image_list):
    num_images = len(image_list)
    num_cols = 10
    num_rows = num_images // num_cols + (num_images % num_cols > 0)

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 10))

    plt.gray() 
    for i, ax in enumerate(axes.flat):
        if i < num_images:
            ax.imshow(image_list[i].reshape(16, 16))
            ax.axis('off')
        else:
            ax.axis('off')

    plt.tight_layout()
    plt.show()

In [11]:
def save_img_centroids(centroids, k):
    i = 1
    for centroid in centroids:
        img = Image.fromarray((centroid.reshape(16,16)).astype(np.uint8))
        directory = f"Results/k{k}/centroid{i}"
        if not os.path.exists(directory):
            os.makedirs(directory)
        img.save(f"Results/k{k}/centroid{i}/centroid{i}.jpg")
        i += 1

In [12]:
def save_img_examples(centroid_id, idx_k, k, num):
    elements = X[idx_k == centroid_id]
    n = len(elements)
    for i in range(num):
        img = Image.fromarray((elements[int((i * n) / num)].reshape(16,16)).astype(np.uint8))
        img.save(f"Results/k{k}/centroid{centroid_id + 1}/example{i + 1}.jpg")

In [13]:
max_iterations = 1000
k_list = {3, 4, 5, 6, 7}
for k in k_list:
    initial_centroids = kMeans_init_centroids(X, k)
    idx, centroids = k_means(X, initial_centroids, max_iterations)
    save_img_centroids(centroids, k)
    for c_id in range(k):
        save_img_examples(c_id, idx, k, 5)